[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/XYSSDFFSDFS)  

# Running NucleicNet on Colab

This note book illustrates a basic benchmark on Google Colab. In general a Google Colab free user will have access to the following computing resource

* CPU. 2-core Intel(R) Xeon(R) @ 2.20GHz Family 6
* RAM. 37G
* GPU. Tesla K80 with 13G memory

We would like to acknowledge [AlphaFold](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb#scrollTo=VzJ5iMjTtoZw) and [ColabFold](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/AlphaFold2_advanced.ipynb#scrollTo=woIxeCPygt7K) in formalising routines in setting up Colab.

# Upload a PDB file
Note that user should make sure that the pdb submitted does not contain non-protein parts and that the protein submitted is intended, even though we have script to retain only the following 20 canonical protein residues: 
`
"ALA","CYS","ASP","GLU","PHE","GLY", 
                    "HIS","ILE","LYS","LEU","MET","ASN", 
                    "PRO","GLN","ARG","SER","THR","VAL", 
                    "TRP","TYR"` as a basic sanitization procedure, we do not replace non-canonical amino acids, which will be left as a hole if present.

Also, do not submit a C-alpha only structure or a structure with no sidechain or heavily stubbed sidechain (particularly common practice in cryoEM!). For simplicity, we also do not do biological assembly and users should look out for rna binding at the interface of assembled unit proteins.

The pdb files has to be put into `../LocalServerExample/` if you are using this notebook. The output is stored in a user designated folder e.g. `../ServerExample/ServerOutput/`

In [ ]:
from google.colab import files
import os

uploaded = files.upload() #@markdown Upload a pdb file of your choice
uploaded_filename = list(uploaded.keys())[0]
print(uploaded_filename)

import os
import shutil
uploaded_suffix = uploaded_filename.split(".")[-1]
shutil.move(uploaded_filename, "upload.%s" %(uploaded_suffix))




# Install Conda and Dependencies

This step regularly takes 1h . A tiny green arrow appears on the line number indicating which line it is stucked at. 
* TODO Specify all dependency with a {}.yaml file s.t. it does not take long to resolve dependency
* TODO Make a NucleicNetLite repo without data to reduce the time to git clone


In [2]:


#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left to download and import third-party software 
#@markdown in this Colab notebook. 

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.


from IPython.utils import io
import os
import subprocess
import tqdm.notebook

# This is for your safety
try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

import jax
if jax.local_devices()[0].platform == 'tpu':
  raise RuntimeError('Colab TPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')
elif jax.local_devices()[0].platform == 'cpu':
  raise RuntimeError('Colab CPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')
else:
  print(f'Running with {jax.local_devices()[0].device_kind} GPU')


#GPU count and name  go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
if IN_COLAB:

  #hard disk space that we can use
  !df -h / | awk '{print $4}'
  #memory that we can use
  !free -h --si | awk  '/Mem:/{print $2}'
  !nvidia-smi -L
  #!nvidia-smi  -q -i 0 -d CLOCK
  #!nvidia-smi  -q -i 0 -d SUPPORTED_CLOCKS
  !nvidia-smi --auto-boost-default=ENABLED -i 0
  !nvidia-smi -pm ENABLED -i 0
  !nvidia-smi -ac 2505,875 -i 0


  %shell pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
  %shell eval "$(conda shell.bash hook)" # copy conda command to shell
  %shell rm -rf /opt/conda # NOTE Dangerous. Luckily mine is a windows.

  # NOTE This get a new anaconda for you
  %shell wget -q -P /tmp \
    https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
      && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
      && rm /tmp/Miniconda3-latest-Linux-x86_64.sh


  PATH=%env PATH
  %env PATH=/opt/conda/bin:{PATH}
  %shell conda update -y conda
  %shell conda init bash
  %shell conda install -y pytorch-lightning=1.5.7 pytorch=1.10.1 -c pytorch -c conda-forge
  %shell conda install -y -c conda-forge nbformat ipywidgets psutil=5.9.0 tqdm  numpy=1.21.2 scipy=1.7.3 pandas=1.3.5 scikit-learn=1.0.2 plotly=5.5.0 seaborn=0.11.2 matplotlib=3.5.1 biopandas=0.2.9
  %shell conda install -y -c anaconda ipywidgets>=7.0.0 nbformat>=4.2.0



  
  %shell which python
  %shell rm -rf NucleicNet
  from google.colab import drive
  drive.mount("/content/drive")
  %shell git clone https://github.com/jhmlam/NucleicNet.git







  # Create a ramdisk to store a database chunk to make Jackhmmer run fast.
  #%shell sudo mkdir -m 777 --parents /tmp/ramdisk
  #%shell sudo mount -t tmpfs -o size=9G ramdisk /tmp/ramdisk



Running with Tesla T4 GPU
Avail
30G
13G
GPU 0: Tesla T4 (UUID: GPU-674bb64b-950c-a081-346f-ad567217d792)
Enabling/disabling default auto boosted clocks is not supported for GPU: 00000000:00:04.0.
Treating as warning and moving on.
All done.
Persistence mode is already Enabled for GPU 00000000:00:04.0.
All done.
Specified clock combination "(MEM 2505, SM 875)" is not supported for GPU 00000000:00:04.0. Run 'nvidia-smi -q -d SUPPORTED_CLOCKS' to see list of supported clock combinations
Treating as warning and moving on.
All done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached google_api_python_client-2.55.0-py2.py3-none-any.whl (8.8 MB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_auth_oauthlib-0.5.2-py2.py3-none-any.whl (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 14.4 MB

# Install NucleicNet with git

In [10]:
%%bash
ls NucleicNet/GoogleColab/
source activate base
cd NucleicNet/NucleicNet/util/ && tar -zxvf feature-3.1.0.tar.gz && cd ../../../
chmod -R 777 NucleicNet/NucleicNet/util/feature-3.1.0/

cd NucleicNet/GoogleColab/
echo -e 'Testing Installation A'
sed -i 's/import scikit-learn/import sklearn/g' command_FinishInstallColabTorch.py
python command_FinishInstallColabTorch.py

echo -e 'Testing Installation B'

cd ../../


command_FinishInstallColabTorch.py
README.md
feature-3.1.0/
feature-3.1.0/tools/
feature-3.1.0/tools/bin/
feature-3.1.0/tools/bin/lisp2model.pl
feature-3.1.0/tools/bin/hits2tab
feature-3.1.0/tools/bin/viewpdb
feature-3.1.0/tools/bin/featurestoarff.py
feature-3.1.0/tools/bin/stat2xml
feature-3.1.0/tools/bin/stat2score
feature-3.1.0/tools/bin/pointfilter.py
feature-3.1.0/tools/bin/ploteval
feature-3.1.0/tools/bin/hits2xml
feature-3.1.0/tools/bin/briefscore.pl
feature-3.1.0/tools/bin/hitfinder.py
feature-3.1.0/tools/bin/mygetsequence.py
feature-3.1.0/tools/bin/pickrandom.py
feature-3.1.0/tools/bin/protein_amber.py
feature-3.1.0/tools/bin/convert_files.py
feature-3.1.0/tools/bin/makeAmberParmsFile.py
feature-3.1.0/tools/bin/atomselector.py
feature-3.1.0/tools/bin/getpdbnr
feature-3.1.0/tools/bin/score2tab
feature-3.1.0/tools/bin/tab2site
feature-3.1.0/tools/bin/viewpdbinfo
feature-3.1.0/tools/bin/normalize.py
feature-3.1.0/tools/bin/nonsitechains.py
feature-3.1.0/tools/bin/site2xml
feature

In [ ]:
# Make sure everything we need is on the path.
import sys
sys.path.append('/content/NucleicNet')

In [ ]:

from distutils.sysconfig import get_python_lib
print(get_python_lib())
import sys
sys.path.append('/content/NucleicNet')



# FAQ